In [1]:
import requests
import pandas as pd
from pprint import pprint

# Question 1

In [2]:
def run_query(q):
    request = requests.post('https://api.thegraph.com/subgraphs/name/uniswap/uniswap-v2'
                            '',
                            json={'query': query})
    if request.status_code == 200:
        return request.json()
    else:
        raise Exception('Query failed. return code is {}.      {}'.format(request.status_code, query))

In [3]:
# Since there are 1000 pairs in Uniswap v2, I have taken first 1000 pairs
query = """

{
  pairs(first:1000, orderBy:volumeUSD, orderDirection: desc) {
    id
    token0 {
      symbol
    }
    token1 {
      symbol
    }
    volumeToken0
    volumeToken1
    volumeUSD
    reserveUSD
    createdAtTimestamp
  }
}
"""
result = run_query(query)


In [4]:
data = []
for d in result['data']['pairs']:
    entry = []
    entry += [d['createdAtTimestamp']]
    if(d['token0']['symbol']=='WETH'):
        entry += ['Buy']
    else:
        entry += ['Sell']
    entry += [d['volumeToken0']]
    entry += [d['volumeToken1']]
    entry += [d['volumeUSD']]
    entry += [d['reserveUSD']]
    entry += [d['id']]
    data += [entry]

In [5]:
df = pd.DataFrame(data,columns=['Timestamp','Side','Base Currency Quantity Traded','Quote Currency Quantity Traded','Volume(USD)','Liquidity in Pool(USD)','Pool ID'])
df.head()

,Timestamp,Side,Base Currency Quantity Traded,Quote Currency Quantity Traded,Volume(USD),Liquidity in Pool(USD),Pool ID
0,1588710145,Sell,28968316075.93534,25428445.957789472315929541,28148301586.03088148507157692398825,240555544.6013326460565655845445066,0xb4e16d0168e52d35cacd2c6185b44281ec28c9dc
1,1589850429,Buy,25294791.652186761577130402,28184939138.219677,27259753396.28834782902435816925784,249256307.9993839356742301364254084,0x0d4a11d5eeaac28ec3f61d100daf4d40471f1852
2,1589164213,Sell,13359878985.090961624810847893,14610718.961628853744310031,13423408989.03659066972801239950601,82645500.35488657281604062818123304,0xa478c2975ab1ea89e8196811f51a7b7ade33eb11
3,1617108509,Sell,6887000,5990652.7371786815366923,11029715648.72159798743878461135205,0.003420674944558147947322700984133829,0x23fe4ee3bd9bfd1152993a7954298bb4d426698f
4,1617139810,Sell,6887000,5974779.3381845163122171,11017636252.92521755536995878068531,0.003421464654088876732141029230008396,0xe5ffe183ae47f1a0e4194618d34c5b05b98953a8


In [6]:
df.to_csv("Uniswap_v2_Data.csv")

# Question 2